In [1]:
import numpy as np 
import pandas as pd
import cvxopt
from cvxopt import matrix
from cvxopt import solvers
import copy
import math

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
arr =[]
for i in range(784):
  arr.append(i)
df_inp = pd.read_csv("/content/drive/MyDrive/train.csv", header=None, usecols =arr)

In [4]:
arr_t = [784]
df_labels = pd.read_csv("/content/drive/MyDrive/train.csv", header=None, usecols=arr_t)

In [5]:
np_X = df_inp.to_numpy()
np_Y = df_labels.to_numpy()
np_X /= 255

In [6]:
def get_data(n1,n2,npxt,npyt):
  c=0
  for k in npyt:
    if k==n1 or k==n2:
      c+=1
  np_X_binary = np.zeros((c,784))
  np_Y_binary = np.zeros(c)
  o=0
  for k in range(np.shape(npyt)[0]):
    if npyt[k]==n1 or npyt[k]==n2:
      np_Y_binary[o]=copy.deepcopy(npyt[k])
      np_X_binary[o]=copy.deepcopy(npxt[k])
      o+=1
  #print(np_Y_binary)
  for i in range(np.shape(np_Y_binary)[0]):
    if np_Y_binary[i]==n1:
      np_Y_binary[i]=1
    else:
      np_Y_binary[i]=-1
  return np_X_binary,np_Y_binary,c

In [ ]:
#Testing
#get_data(4,6,np_X,np_Y)

Setting up training of 1v1 classifiers

In [7]:
gamma = 0.05
C=1.0

In [8]:
def kernel(np_arr0,np_arr1,gama):
  ut = math.exp(-1*gama*((np.linalg.norm((np_arr0-np_arr1),ord=None))**2))
  return ut

In [9]:
def matrices(m,Co,np_x,np_y,gama):
  q_temp = np.ones((m,1), dtype=np.float64)
  q_temp *= (-1)
  a_temp = copy.deepcopy(np_y)
  a_temp = a_temp.reshape(1,m)
  a_temp = a_temp.astype(np.float64)
  b_temp= 0.0
  h_temp = np.zeros((2*m,1), dtype=np.float64)
  for j in range(m):
    h_temp[j]=Co
  idi = np.identity(m,dtype = np.float64)
  idi_neg = copy.deepcopy(idi)
  idi_neg *= (-1)
  g_temp = np.concatenate((idi,idi_neg),axis=0)
  p_temp = np.zeros((m,m), dtype=np.float64)
  for i in range(m):
    for j in range(i+1):
      use = (np_y[i].item()*np_y[j].item())*(kernel(np_x[i],np_x[j],gama))
      p_temp[i][j]=use
      p_temp[j][i]=use
  return p_temp,q_temp,g_temp,h_temp,a_temp,b_temp

In [10]:
def solver_matrices(p_temp,q_temp,g_temp,h_temp,a_temp,b_temp):
  Pt = matrix(p_temp)
  qt =matrix(q_temp)
  Gt =matrix(g_temp)
  ht =matrix(h_temp)
  At =matrix(a_temp)
  bt =matrix(b_temp)
  return Pt,qt,Gt,ht,At,bt

In [11]:
def computation(alpha_kern,npYfunc,np_X_func,Xj,nums,gama):
  s=0.0
  for i in range(nums):
    s += (alpha_kern[i])*(npYfunc[i].item())*kernel(np_X_func[i],Xj,gama)
  return s

In [12]:
def bias(npx,npy,m,alphaa,gama):
  b = 0.0
  maxo= -1000000.0
  mino= 1000000.0
  for i in range(m):
    cont = computation(alphaa,npy,npx,npx[i],m,gama)
    if npy[i].item()==1:
      mino = min(mino,cont)
    else:
      maxo = max(maxo,cont)
  b += maxo
  b+= mino 
  b /= 2.0
  return b


Training of 1v1 classifiers

In [13]:
alpha_all = np.zeros((45,4500,1),dtype=np.float64)
b_all = np.zeros((45,1),dtype=np.float64)
all_datas = {}
ind =0 
for i in range(10):
  for j in range(i+1,10):
    npxd,npyd,md = get_data(i,j,np_X,np_Y)
    all_datas[(i,j)]= (npxd,(npyd,md))
    pt,qt,gt,ht,at,bt = matrices(md,C,npxd,npyd,gamma)
    P,q,G,h,A,b = solver_matrices(pt,qt,gt,ht,at,bt)
    sol = solvers.qp(P,q,G,h,A,b)
    alph = sol['x']
    alpha_all[ind]= alph
    b_all[ind]= bias(npxd,npyd,md,alph,gamma)
    print(ind)
    ind += 1

     pcost       dcost       gap    pres   dres
 0: -1.6492e+02 -6.6070e+03  3e+04  2e+00  2e-15
 1: -1.1407e+02 -3.0686e+03  5e+03  2e-01  3e-15
 2: -1.0459e+02 -7.6952e+02  9e+02  3e-02  3e-15
 3: -1.2684e+02 -3.0203e+02  2e+02  6e-03  3e-15
 4: -1.4133e+02 -2.0101e+02  6e+01  1e-03  2e-15
 5: -1.4945e+02 -1.6809e+02  2e+01  2e-04  2e-15
 6: -1.5144e+02 -1.6280e+02  1e+01  3e-05  2e-15
 7: -1.5364e+02 -1.5728e+02  4e+00  7e-06  2e-15
 8: -1.5443e+02 -1.5550e+02  1e+00  1e-14  2e-15
 9: -1.5478e+02 -1.5492e+02  1e-01  2e-15  2e-15
10: -1.5483e+02 -1.5484e+02  4e-03  5e-15  2e-15
11: -1.5483e+02 -1.5483e+02  6e-05  1e-14  2e-15
Optimal solution found.
0
     pcost       dcost       gap    pres   dres
 0: -3.7366e+02 -8.4330e+03  4e+04  2e+00  2e-15
 1: -2.8341e+02 -4.5691e+03  7e+03  2e-01  2e-15
 2: -2.6181e+02 -1.1549e+03  1e+03  3e-02  3e-15
 3: -3.1375e+02 -5.8168e+02  3e+02  6e-03  2e-15
 4: -3.4227e+02 -4.1865e+02  8e+01  1e-03  2e-15
 5: -3.5409e+02 -3.7849e+02  2e+01  2e-04  2e

In [14]:
print(np_Y)

[[5.]
 [1.]
 [7.]
 ...
 [3.]
 [8.]
 [7.]]


In [15]:
alpha_cop = copy.deepcopy(alpha_all)
b_cop = copy.deepcopy(b_all)

In [16]:
print(alpha_all)

[[[8.95309077e-09]
  [8.78393612e-09]
  [1.03172394e-08]
  ...
  [5.62194884e-09]
  [4.61059766e-09]
  [9.99999977e-01]]

 [[2.86702874e-09]
  [3.14855045e-08]
  [4.28883583e-09]
  ...
  [3.20598313e-09]
  [1.23857775e-06]
  [1.28754060e-01]]

 [[4.20730029e-08]
  [4.05348060e-08]
  [1.42794175e-08]
  ...
  [3.84118828e-08]
  [1.13389206e-08]
  [3.89480248e-01]]

 ...

 [[1.30561481e-09]
  [3.28186674e-09]
  [4.57651788e-09]
  ...
  [8.35322975e-09]
  [1.35701008e-08]
  [7.00239305e-01]]

 [[1.06883460e-08]
  [1.39762674e-08]
  [1.96004417e-08]
  ...
  [2.06881755e-08]
  [1.80291276e-01]
  [5.51677193e-01]]

 [[1.57998249e-09]
  [3.12961638e-09]
  [1.16932411e-01]
  ...
  [2.12743463e-01]
  [2.72543879e-09]
  [4.66763683e-09]]]


In [17]:
index = {}
use=0
for i in range(10):
  for j in range(i+1,10):
    index[(i,j)]=use
    use+=1

In [18]:
print(index)

{(0, 1): 0, (0, 2): 1, (0, 3): 2, (0, 4): 3, (0, 5): 4, (0, 6): 5, (0, 7): 6, (0, 8): 7, (0, 9): 8, (1, 2): 9, (1, 3): 10, (1, 4): 11, (1, 5): 12, (1, 6): 13, (1, 7): 14, (1, 8): 15, (1, 9): 16, (2, 3): 17, (2, 4): 18, (2, 5): 19, (2, 6): 20, (2, 7): 21, (2, 8): 22, (2, 9): 23, (3, 4): 24, (3, 5): 25, (3, 6): 26, (3, 7): 27, (3, 8): 28, (3, 9): 29, (4, 5): 30, (4, 6): 31, (4, 7): 32, (4, 8): 33, (4, 9): 34, (5, 6): 35, (5, 7): 36, (5, 8): 37, (5, 9): 38, (6, 7): 39, (6, 8): 40, (6, 9): 41, (7, 8): 42, (7, 9): 43, (8, 9): 44}


In [19]:
print(b_all[index[(4,5)]])

[-0.1504886]


In [20]:
# test set accuracy
df_inp_test = pd.read_csv("/content/drive/MyDrive/test.csv", header=None, usecols =arr)
df_labels_test = pd.read_csv("/content/drive/MyDrive/test.csv", header=None, usecols=arr_t)

In [21]:
np_X_test = df_inp_test.to_numpy()
np_Y_test = df_labels_test.to_numpy()
print(np_X_test)
print(np_Y_test)
np_X_test /= 255

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0.]
 [2.]
 [1.]
 ...
 [0.]
 [3.]
 [2.]]


In [33]:
points = np.shape(np_Y_test)[0]
correct = 0
for o in range(300):
  num_votes = np.zeros(10)
  for i in range(10):
    for j in range(i+1,10):
      op = index[(i,j)]
      num =0.0
      num += b_all[op].item()
      npx = all_datas[(i,j)][0]
      npy = all_datas[(i,j)][1][0]
      mi = all_datas[(i,j)][1][1]
      num += computation(alpha_all[op],npy,npx,np_X_test[o],mi,gamma)
      #print(num)
      if num>=0.0:
        num_votes[i]+=1
      else:
        num_votes[j]+=1
  maxo = max(num_votes)
  #print(num_votes)
  #print(np_Y_test[o])
  pred = np.where(num_votes==maxo)
  predo= pred[0][0]
  #print(type(predo))
  #print(type(pred))
  #print(predo)
  if (predo)==int(np_Y_test[o]):
    correct+=1
  #print(correct)
print(correct/300)


0.5633333333333334


In [34]:
print(alpha_all[index[(4,5)]])

[[5.95247019e-02]
 [9.04632193e-08]
 [1.08737736e-01]
 ...
 [2.16872218e-08]
 [2.15061483e-08]
 [1.05757846e-08]]
